# Домашнє завдання: Прогнозування орендної плати за житло

## Мета завдання
Застосувати знання з лекції для побудови моделі лінійної регресії, що прогнозує орендну плату за житло в Індії. Ви пройдете весь цикл вирішення задачі машинного навчання: від дослідницького аналізу до оцінки якості моделі.

## Опис датасету
**House Rent Prediction Dataset** містить інформацію про 4700+ оголошень про оренду житла в Індії з такими параметрами:
- **BHK**: Кількість спалень, залів, кухонь
- **Rent**: Орендна плата (цільова змінна)
- **Size**: Площа в квадратних футах
- **Floor**: Поверх та загальна кількість поверхів
- **Area Type**: Тип розрахунку площі
- **Area Locality**: Район
- **City**: Місто
- **Furnishing Status**: Стан меблювання
- **Tenant Preferred**: Тип орендаря
- **Bathroom**: Кількість ванних кімнат
- **Point of Contact**: Контактна особа

---

## Завдання 1: Завантаження та перший огляд даних (1 бал)

**Що потрібно зробити:**
1. Завантажте дані з файлу `House_Rent_Dataset.csv`
2. Виведіть розмір датасету
3. Покажіть перші 5 рядків
4. Виведіть загальну інформацію про дані (включно з типами даних та кількістю значень)


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


In [2]:
df = pd.read_csv('../data/House_Rent_Dataset.csv')
df.shape

(4746, 12)

In [3]:
df.head(5)

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


## Завдання 2: Дослідницький аналіз даних (EDA) (5 балів)

**Що потрібно зробити:**
1. **Аналіз пропущених значень.** Перевірте наявність і відсоток пропущених значень у кожній колонці
2. **Базова статистика.** Обчисліть базову статистику (середнє, квартилі, стандартне відхилення) для числових змінних.
3. **Аналіз цільової змінної.** Побудуйте гістограму розподілу цільової змінної (Rent)
4. **Робота з викидами.** Знайдіть та видаліть викиди в цільовій змінній (якщо є). Визначити викиди можна будь-яким зрозумілим для вас способом, як варіант - таким, що використовується в побудові box-plot (https://en.wikipedia.org/wiki/Box_plot#Example_with_outliers).
5. **Аналіз категоріальних змінних.** Виведіть кількість унікальних значень для кожної з категоріальних колонок.


In [5]:
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_percent

Posted On            0.0
BHK                  0.0
Rent                 0.0
Size                 0.0
Floor                0.0
Area Type            0.0
Area Locality        0.0
City                 0.0
Furnishing Status    0.0
Tenant Preferred     0.0
Bathroom             0.0
Point of Contact     0.0
dtype: float64

In [6]:
stats = df[['BHK', 'Rent','Size', 'Bathroom']].describe()
stats.round(2)

,BHK,Rent,Size,Bathroom
count,4746.00,4746.00,4746.00,4746.00
mean,2.08,34993.45,967.49,1.97
std,0.83,78106.41,634.20,0.88
min,1.00,1200.00,10.00,1.00
25%,2.00,10000.00,550.00,1.00
50%,2.00,16000.00,850.00,2.00
75%,3.00,33000.00,1200.00,2.00
max,6.00,3500000.00,8000.00,10.00


- BHK: більшість квартир мають 2 BHK, розподіл має правий хвіст до 6.
- Rent: широкий розподіл із сильною правосторонньою асиметрією (середнє значення значно перевищує медіану) та значними викидами (стандартне відхилення перевищує середнє).  
- Size: широкий розподіл площі, більшість квартир зосереджені в нижньому діапазоні, присутні великі аутлаєри.  
- Bathroom: основна маса значень зосереджена навколо 1–2, присутні поодинокі високі значення.  


In [7]:
fig = px.histogram(
    df,
    x='Rent',
    nbins=100,
    title='Розподіл цільової змінної (Орендна плата)',
    labels={'Rent': 'Орендна плата'}
)

fig.update_yaxes(title='Кіл-ть оголошень')

fig.update_layout(
    showlegend=False,
    height=400
)

fig.show()

Розподіл цільової змінної має сильну правосторонню асиметрію з наявністю екстремальних значень, які суттєво впливають на масштаб графіка та ускладнюють інтерпретацію основної маси даних.

In [8]:
def remove_outliers(df, column):
    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)

    IQR = Q3-Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return filtered_df

df_clean = remove_outliers(df, 'Rent')

In [9]:
df_base = df_clean.copy()

In [10]:
fig = px.histogram(
    df_clean,
    x='Rent',
    nbins=30,
    title='Розподіл цільової змінної (Орендна плата) після видалення викидів',
    labels={'Rent': 'Орендна плата'}
)

fig.update_yaxes(title='Кіл-ть оголошень')

fig.update_layout(
    showlegend=False,
    height=400
)

fig.show()

Після видалення викидів розподіл орендної плати став більш читабельним: основна маса значень зосереджена в діапазоні до 70 000, при цьому зберігається правостороння асиметрія розподілу.


In [11]:
stats_2 = df_clean[['BHK', 'Rent','Size', 'Bathroom']].describe()
stats_2.round(2)

,BHK,Rent,Size,Bathroom
count,4226.00,4226.00,4226.00,4226.00
mean,1.96,19286.16,871.78,1.81
std,0.75,13825.40,485.78,0.71
min,1.00,1200.00,10.00,1.00
25%,1.00,9500.00,520.00,1.00
50%,2.00,15000.00,800.00,2.00
75%,2.00,25000.00,1100.00,2.00
max,6.00,67000.00,4200.00,7.00


Після очищення даних базові статистичні показники стали більш стабільними та реалістичними: середнє значення та стандартне відхилення зменшилися,що свідчить про зниження впливу екстремальних значень на розподіл.


In [12]:
df[['Posted On', 'Floor', 'Area Type', 'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']].nunique()

Posted On              81
Floor                 480
Area Type               3
Area Locality        2235
City                    6
Furnishing Status       3
Tenant Preferred        3
Point of Contact        3
dtype: int64

Змінні `Area Type, City, Furnishing Status, Tenant Preferred та Point of Contact` містять невелику кількість категорій і є зручними для подальшого кодування.


## Завдання 3: Аналіз кореляцій та взаємозв'язків (3 бали)

**Що потрібно зробити:**
1. Обчисліть матрицю кореляцій для числових змінних
2. Візуалізуйте кореляційну матрицю за допомогою heatmap
3. Побудуйте scatter plot між Size та Rent
4. Проаналізуйте взаємозв'язок між BHK та Rent за допомогою boxplot (який розподіл плати для різних значень BHK)


In [13]:
metrics_df = df[['BHK', 'Rent', 'Size', 'Bathroom']]
correlation_matrix = metrics_df.corr()

fig = px.imshow(
    correlation_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Кореляція між числовими змінними',
    labels=dict(color="Кореляція")
)
fig.update_layout(height=500)
fig.show()

З кореляційної матриці видно, що цільова змінна Rent має помірну позитивну кореляцію з числовими ознаками BHK, Size та Bathroom.  
Водночас між самими незалежними змінними спостерігається висока кореляція.


In [14]:
fig = px.scatter(
    df_clean,
    x='Size',
    y='Rent',
    title='Rent vs Size',
    labels={'Size': 'Площа (кв.футів)', 'Rent': 'Орендна плата'},
    opacity=0.6
)

fig.update_layout(height=500)
fig.show()

Між Size та Rent спостерігається помірний позитивний зв’язок, однак залежність не є чітко лінійною. На графіку помітні кілька груп спостережень, де для подібних значень площі орендна плата суттєво відрізняється.  
Це може свідчити про вплив інших факторів (наприклад, міста, району, меблювання або кількості кімнат), які не враховані в даній візуалізації.


In [15]:
fig = px.box(
    df_clean,
    x='BHK', 
    y='Rent', 
    width=800, 
    height=600,
    points='outliers'
)

fig.show()

Boxplot показує, що зі збільшенням кількості BHK загалом зростає медіанна орендна плата.  
- Для квартир з 1–2 BHK спостерігається наявність значних
викидів у верхній частині розподілу, що свідчить про вплив інших факторів, окрім кількості кімнат.  
- Для квартир з 3 BHK характерний широкий розкид значень орендної плати.  
- Квартири з 4–5 BHK характеризуються вищою медіанною орендною платою та меншою варіативністю порівняно з іншими категоріями.  
- Для категорії 6 BHK медіана є нижчою, ніж для деяких менших значень BHK, що може бути пов’язано з невеликою кількістю спостережень або неоднорідністю цієї групи.


## Завдання 4: Feature Engineering та підготовка даних (4 бали)

**Що потрібно зробити:**
1. Закодуйте категоріальні змінні за допомогою One-Hot Encoding. Пригадайте, що в лекції ми говорили щодо кодування кат. змінних з великої кількістю різних значень і як працювати з такими випадками. Ви можете закодувати не всі кат. змінні, а лише ті, що вважаєте за потрібні (скажімо ті, що мають відносно небагато різних значень).
2. **Опціонально (по 0.5 бала за кожну доцільну ознаку):** Додайте нові ознаки, обчислені на основі наявних даних, які б на ваш погляд були корисними для моделі
3. Виберіть ознаки для побудови моделі (виключіть непотрібні колонки). Виключити можна, наприклад, ті колонки, які мають категоріальний тип і забагато (більше 20) різних значень. Треба виключити хоча б 1 колонку.
4. Розділіть дані на ознаки (X) та цільову змінну (y)
5. Застосуйте стандартизацію до числових ознак


In [16]:
#Кодуємо категоріальні змінні

dummies = pd.get_dummies(
    df_clean[['Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']]
)

df_clean = pd.concat([df_clean, dummies], axis=1)


In [17]:
# Витягуємо поверхи з колонки "Floor" та створюємо дві нові колонки

df_floors = df_clean['Floor'].str.split(' out of ', expand=True)
df_floors = df_floors.rename(columns={
    0: 'appart_floor',
    1: 'total_floors'
})
df_floors = df_floors.replace('Ground', '0')
df_floors[df_floors.isna().any(axis=1)]

,appart_floor,total_floors
2553,3,None
2883,0,None
4490,1,None
4560,1,None


In [18]:
df_clean = pd.concat([df_clean, df_floors], axis=1)

In [19]:
df_clean = df_clean.dropna()

In [20]:
# Деякі значення колонки "appart_floor" були типу str, знаходимо ці значення та видаляємо

df_clean.loc[
    ~df_clean['appart_floor'].astype(str).str.fullmatch(r'\d+'),
    'appart_floor'
].value_counts()

appart_floor
Upper Basement    18
Lower Basement    10
Name: count, dtype: int64

In [21]:
df_clean = df_clean[~df_clean['appart_floor'].isin(['Upper Basement', 'Lower Basement'])]
df_clean[['appart_floor', 'total_floors']] = df_clean[['appart_floor', 'total_floors']].astype(int)

In [22]:
# Витягуємо місяць публікації оголошень

df_clean['Posted On'] = pd.to_datetime(df_clean['Posted On'])
df_clean['posted_month'] = df_clean['Posted On'].dt.month
df_clean['posted_month'].value_counts().sort_values()


posted_month
4     220
7     797
5    1548
6    1629
Name: count, dtype: int64

In [23]:
fig = px.box(
    df_clean,
    x='posted_month', 
    y='Rent', 
    width=500, 
    height=400,
    points='outliers'
)

fig.show()

Хоча у липні спостерігається підвищення медіани та більший розкид орендної плати, загальної чіткої сезонної динаміки у межах доступного періоду спостережень не видно.  

In [24]:
df_clean.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred',
       'Bathroom', 'Point of Contact', 'Area Type_Built Area',
       'Area Type_Carpet Area', 'Area Type_Super Area', 'City_Bangalore',
       'City_Chennai', 'City_Delhi', 'City_Hyderabad', 'City_Kolkata',
       'City_Mumbai', 'Furnishing Status_Furnished',
       'Furnishing Status_Semi-Furnished', 'Furnishing Status_Unfurnished',
       'Tenant Preferred_Bachelors', 'Tenant Preferred_Bachelors/Family',
       'Tenant Preferred_Family', 'Point of Contact_Contact Agent',
       'Point of Contact_Contact Builder', 'Point of Contact_Contact Owner',
       'appart_floor', 'total_floors', 'posted_month'],
      dtype='object')

In [25]:
metrics_df_clean = df_clean.select_dtypes(include='number')
correlation_matrix_clean = metrics_df_clean.corr()

fig = px.imshow(
    correlation_matrix_clean,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Кореляція між числовими змінними',
    labels=dict(color="Кореляція")
)
fig.update_layout(height=500)
fig.show()

З кореляційної матриці видно, що найбільший зв’язок з орендною платою мають такі ознаки: total_floors, Bathroom, appart_floor, BHK та Size.  
При цьому жодна числова змінна не має дуже високої кореляції з Rent (0.7+), що означає: вартість оренди формується під впливом сукупності факторів, а не однієї домінуючої характеристики.

In [26]:
metrics_df_clean = metrics_df_clean.drop(columns='Rent')


In [27]:
features = metrics_df_clean.columns.tolist() + dummies.columns.tolist()
features

['BHK',
 'Size',
 'Bathroom',
 'appart_floor',
 'total_floors',
 'posted_month',
 'Area Type_Built Area',
 'Area Type_Carpet Area',
 'Area Type_Super Area',
 'City_Bangalore',
 'City_Chennai',
 'City_Delhi',
 'City_Hyderabad',
 'City_Kolkata',
 'City_Mumbai',
 'Furnishing Status_Furnished',
 'Furnishing Status_Semi-Furnished',
 'Furnishing Status_Unfurnished',
 'Tenant Preferred_Bachelors',
 'Tenant Preferred_Bachelors/Family',
 'Tenant Preferred_Family',
 'Point of Contact_Contact Agent',
 'Point of Contact_Contact Builder',
 'Point of Contact_Contact Owner']

In [28]:
X = df_clean[features]
y = df_clean['Rent']

print(f"\nРозмір X (ознак): {X.shape}")
print(f"Розмір y (цілі): {y.shape}")


Розмір X (ознак): (4194, 24)
Розмір y (цілі): (4194,)


In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

## Завдання 5: Розділення даних та навчання моделі (3 бали)

**Що потрібно зробити:**
1. Розділіть дані на навчальну (80%) та тестову (20%) вибірки.
2. Створіть модель лінійної регресії.
3. Навчіть модель на навчальних даних.
4. Виведіть усі коефіцієнти моделі (ваги) та напишіть, які 2 ознаки найбільше впливають на прогноз.
5. Зробіть прогнози на тренувальній та тестовій вибірках.

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y,
    test_size=0.2,
    random_state=42
)

In [31]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [32]:
for feature, weight in zip(model.feature_names_in_, model.coef_):
    print(f"{feature}: {weight:.2f}")

print(f"\nЗміщення (intercept): {model.intercept_:.2f}")

BHK: 2429.60
Size: 3569.40
Bathroom: 1765.46
appart_floor: 118.41
total_floors: 1635.95
posted_month: 265.20
Area Type_Built Area: -30.38
Area Type_Carpet Area: 24.32
Area Type_Super Area: -22.98
City_Bangalore: -716.75
City_Chennai: -1043.00
City_Delhi: 200.71
City_Hyderabad: -1604.54
City_Kolkata: -1494.20
City_Mumbai: 5083.09
Furnishing Status_Furnished: 1191.08
Furnishing Status_Semi-Furnished: -80.46
Furnishing Status_Unfurnished: -714.57
Tenant Preferred_Bachelors: 308.86
Tenant Preferred_Bachelors/Family: 70.73
Tenant Preferred_Family: -517.74
Point of Contact_Contact Agent: 1723.32
Point of Contact_Contact Builder: -18.10
Point of Contact_Contact Owner: -1722.12

Зміщення (intercept): 19257.39


In [33]:
y_train_pred = model.predict(X_train)

y_test_pred = model.predict(X_test)

comparison = pd.DataFrame({
    'Реальна ціна оренди': y_test.values[:10],
    'Прогнозована ціна оренди': y_test_pred[:10].round(0),
    'Помилка': (y_test.values[:10] - y_test_pred[:10]).round(0)
})
print("Приклади прогнозів на тестовій вибірці:")
print(comparison)

Приклади прогнозів на тестовій вибірці:
   Реальна ціна оренди  Прогнозована ціна оренди  Помилка
0                11500                   14423.0  -2923.0
1                 6500                    8221.0  -1721.0
2                 6500                    4124.0   2376.0
3                43000                   39985.0   3015.0
4                 6500                    8312.0  -1812.0
5                 7000                   17091.0 -10091.0
6                 4000                    6446.0  -2446.0
7                 6000                    5195.0    805.0
8                 8000                   11391.0  -3391.0
9                 7500                    9099.0  -1599.0


## Завдання 6: Оцінка якості моделі (2 бали)

**Що потрібно зробити:**
1. Обчисліть MAE, RMSE та R² для навчальної та тестової вибірок
2. Порівняйте метрики та зробіть висновок про якість моделі
3. Проаналізуйте і дайте висновок, чи є ознаки перенавчання або недонавчання (**Нагадування**: перенавчання - коли модель дуже добре працює на тренувальних даних, але погано на тестових; недонавчання - коли модель погано працює навіть на тренувальних даних)
4. Побудуйте графік розсіювання "реальні vs прогнозовані значення" та зробіть висновок про якість моделі


In [34]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_test_pred)

print("="*50)
print("МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):")
print("="*50)
print(f"\nMAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

mae = mean_absolute_error(y_train, y_train_pred)
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)
r2_train = r2_score(y_train, y_train_pred)

print("="*50)
print("МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тренувальній вибірці):")
print("="*50)
print(f"\nMAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2_train:.3f}")

МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):

MAE: 5389.38
RMSE: 7606.08
R²: 0.693
МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тренувальній вибірці):

MAE: 5402.29
RMSE: 7537.24
R²: 0.703


Модель демонструє схожі значення метрик як на тренувальній, так і на тестовій вибірках, що свідчить про відсутність ознак перенавчання або недонавчання.  

Значення МАЕ становить близько 5400 і може вважатися помірною похибкою. Значення R² близьке до 0.7 і свідчить про хорошу узгодженість моделі з даними. Модель пояснює близько 70% варіації орендної плати за рахунок використаних ознак. 

In [35]:
fig = px.scatter(
    x=y_test,
    y=y_test_pred,
    title='Реальна vs Прогнозована ціна оренди (тестова вибірка)',
    labels={'x': 'Реальна ціна оренди', 'y': 'Прогнозована ціна оренди'},
    opacity=0.6
)

max_val = max(y_test.max(), y_test_pred.max())
fig.add_trace(
    go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        name='Ідеальний прогноз',
        line=dict(color='red', dash='dash')
    )
)

fig.update_layout(height=500)
fig.show()

Судячи з графіка, модель має тенденцію до переоцінки дешевших квартир та недооцінки дорожчих. Точки загалом слідують лінії ідеального прогнозу, однак для високих значень оренди спостерігається більший розкид, що вказує на обмеження моделі та можливу нестачу важливих ознак. 

## Завдання 7: Аналіз помилок (4 бали)

**Що потрібно зробити:**
1. Обчисліть помилки (residuals = реальні - прогнозовані значення)
2. Побудуйте гістограму розподілу помилок
3. Створіть scatter plot помилок відносно величини прогнозованих значень. Чи росте помилка з ростом прогнозованого значення?
4. Знайдіть 5 прогнозів з найбільшими помилками
5. Проаналізуйте, на яких типах житла модель помиляється найбільше. Типи можна розрізняти за кількістю кімнат чи містом, наприклад.
6. Подумайте і напишіть, які наступні кроки ви б зробили, аби поліпшити якість моделі. Опціонально можна їх зробити і ми перевіримо :)

In [36]:
residuals = y_test - y_test_pred

fig = px.histogram(
    x=residuals,
    nbins=50,
    title='Розподіл помилок прогнозування',
    labels={'x': 'Помилка (реальні - прогнозовані)', 'count': 'Кількість'},
    color_discrete_sequence=['#e74c3c']
)
fig.add_vline(x=0, line_dash="dash", line_color="black", annotation_text="Ідеальний прогноз")
fig.update_layout(height=400)
fig.show()

Розподіл помилок зосереджений навколо нуля, що свідчить про відсутність суттєвого систематичного зміщення прогнозів.   Розподіл є близьким до симетричного. Основна маса помилок зосереджена в діапазоні приблизно ±5–7 тис., однак присутні поодинокі великі відхилення (до ±30 тис.), що вказує на складність прогнозування для окремих об’єктів, зокрема дорогого сегмента.

In [37]:
fig = px.scatter(
    x=y_test_pred,
    y=residuals,
    title='Залежність помилок від прогнозованих значень',
    labels={'x': 'Прогнозована ціна оренди', 'y': 'Помилка'},
    opacity=0.5
)

fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Без помилки")

fig.update_layout(height=400)
fig.show()

На графіку видно, що зі зростанням прогнозованої орендної плати зростає розкид помилок, що вказує на обмеження моделі при прогнозуванні вищої оренди. Водночас помилки зосереджені навколо нуля, що підтверджує відсутність значного систематичного зміщення. 

In [38]:
errors_df = pd.DataFrame({
    'real': y_test.values,
    'predicted': y_test_pred,
    'error': np.abs(residuals)
})

top_errors = errors_df.nlargest(5, 'error')
print("Квартири з найбільшими помилками прогнозування:")
print(top_errors)

Квартири з найбільшими помилками прогнозування:
       real     predicted         error
1272  65000  30733.591148  34266.408852
1090  65000  32959.252374  32040.747626
840   65000  33133.843112  31866.156888
2087  62000  31045.417118  30954.582882
1836  60000  29317.998376  30682.001624


Модель найбільше помиляється для дорогих квартир, систематично недооцінюючи їхню орендну плату.

In [39]:
errors_full = df_clean[['BHK', 'City', 'Furnishing Status']].loc[y_test.index].copy()

errors_full['predicted'] = y_test_pred
errors_full['abs_error'] = np.abs(residuals)
errors_full.head(3)

,BHK,City,Furnishing Status,predicted,abs_error
3767,2,Chennai,Semi-Furnished,14422.568342,2922.568342
1733,1,Bangalore,Semi-Furnished,8220.968129,1720.968129
189,1,Kolkata,Furnished,4123.925493,2376.074507


In [40]:
errors_full.groupby('BHK')['abs_error'].mean().sort_values(ascending=False)

BHK
4    14567.238142
6    11546.735936
3     7668.393296
2     4768.969979
1     4594.873834
5     1153.478033
Name: abs_error, dtype: float64

In [41]:
errors_full.groupby('City')['abs_error'].mean().sort_values(ascending=False)

City
Mumbai       9406.531336
Delhi        6207.540249
Hyderabad    4887.528582
Bangalore    4321.367817
Kolkata      4265.111533
Chennai      4252.393706
Name: abs_error, dtype: float64

In [42]:
errors_full.groupby('Furnishing Status')['abs_error'].mean().sort_values(ascending=False)

Furnishing Status
Furnished         6542.592699
Semi-Furnished    5497.640924
Unfurnished       4902.079310
Name: abs_error, dtype: float64

**Аналіз середньої абсолютної помилки показав:**  
- Модель найбільше помиляється для квартир з 4–6 BHK, що свідчить про складність прогнозування цін для великих та, ймовірно, преміальних обʼєктів. Це може бути пов'язано з меншою кількістю таких об'єктів в наших даних, що ускладнює навчання моделі для преміального сегменту.  
- За містами найбільші помилки спостерігаються в Mumbai та Delhi, де ринок оренди є більш різнорідний (можуть суттєво відрізнятися ціни залежно від району, інфрастукртури чи класу житла).  
- Також вищі помилки характерні для повністю та напівмебльованих квартир, що може бути повʼязано з відсутністю детальної інформації про якість та стан меблів, ремонту та рівня комплектації, які суттєво впливають на фінальну ціну, але не враховані в моделі.  


**Загалом аналіз помилок підтверджує, що модель краще працює для типових об'єктів середнього сегмента і менш стабільна для преміальних або більш варіантивних категорій житла.**

Для покращення якості моделі в подальшому доцільно, наприклад, застосувати логарифмічне перетворення цільової змінної, що дозволить зменшити вплив екстремальних значен, які спостерігались для дорогих квартир. Також модель могла б виграти від додавання більш детальної інформації про локацію, зокрема район розташування житла. Окрім цього, могло б допомогти використання інших моделей, які могли б краще врахувати нелінійні залежності між ознаками та підвищити стабільність прогнозів для об’єктів з високою орендною платою.

### Завдання 7.6. Спроба покращити якість моделі

Додамо ще ознаки до вже наявних:  
- Закодуємо Area Locality;
- Додамо колонки з взаємодією декількох ознак.

In [43]:
top_localities = df_clean['Area Locality'].value_counts().head(13).index
df_clean['localities_grouped'] = df_clean['Area Locality'].apply(
    lambda x: x if x in top_localities else 'other'
)
locality_dummies = pd.get_dummies(df_clean['localities_grouped'], prefix='locality')

In [44]:
df_clean = pd.concat([df_clean, locality_dummies], axis=1)

In [45]:
features_enh = features + locality_dummies.columns.tolist()
X_enh = df_clean[features_enh].copy()

X_enh['bathroom_per_bhk'] = df_clean['Bathroom'] / df_clean['BHK']
X_enh['size_per_bhk'] = df_clean['Size'] / df_clean['BHK']
X_enh['floor_ratio'] = df_clean['appart_floor'] / df_clean['total_floors']

for city in ['City_Mumbai', 'City_Delhi']:
    if city in X_enh.columns:
        X_enh[f'Size_x_{city}'] = df_clean['Size'] * X_enh[city]

for furn in ['Furnishing Status_Furnished', 'Furnishing Status_Semi-Furnished']:
    if furn in X_enh.columns:
        X_enh[f'Size_x_{furn}'] = df_clean['Size'] * X_enh[furn]

if 'City_Mumbai' in X_enh.columns and 'Furnishing Status_Furnished' in X_enh.columns:
    X_enh['Mumbai_x_Furnished'] = X_enh['City_Mumbai'] * X_enh['Furnishing Status_Furnished']

In [46]:
X = X_enh
y = df_clean['Rent']

print(f"\nРозмір X (ознак): {X.shape}")
print(f"Розмір y (цілі): {y.shape}")


Розмір X (ознак): (4194, 46)
Розмір y (цілі): (4194,)


In [47]:
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [48]:
for feature, weight in zip(model.feature_names_in_, model.coef_):
    print(f"{feature}: {weight:.2f}")

print(f"\nЗміщення (intercept): {model.intercept_:.2f}")

BHK: 1645.84
Size: 3080.11
Bathroom: 2297.93
appart_floor: 90.82
total_floors: 1233.35
posted_month: 330.38
Area Type_Built Area: -28.58
Area Type_Carpet Area: 58.94
Area Type_Super Area: -57.68
City_Bangalore: -160.71
City_Chennai: -491.06
City_Delhi: 1161.13
City_Hyderabad: -1292.64
City_Kolkata: -1094.69
City_Mumbai: 2142.56
Furnishing Status_Furnished: 591.15
Furnishing Status_Semi-Furnished: -125.28
Furnishing Status_Unfurnished: -267.96
Tenant Preferred_Bachelors: 312.71
Tenant Preferred_Bachelors/Family: 104.27
Tenant Preferred_Family: -574.50
Point of Contact_Contact Agent: 1670.49
Point of Contact_Contact Builder: -23.17
Point of Contact_Contact Owner: -1669.13
locality_Banjara Hills, NH 9: 421.94
locality_Chromepet, GST Road: -51.70
locality_Electronic City: -133.29
locality_Gachibowli: 995.45
locality_Iyyappanthangal: 96.18
locality_K R Puram: -186.08
locality_Kondapur: 41.84
locality_Laxmi Nagar: -635.61
locality_Madipakkam: -188.39
locality_Medavakkam: -139.78
locality_Miy

In [49]:
y_train_pred = model.predict(X_train)

y_test_pred = model.predict(X_test)

comparison = pd.DataFrame({
    'Реальна ціна оренди': y_test.values[:10],
    'Прогнозована ціна оренди': y_test_pred[:10].round(0),
    'Помилка': (y_test.values[:10] - y_test_pred[:10]).round(0)
})
print("Приклади прогнозів на тестовій вибірці:")
print(comparison)

Приклади прогнозів на тестовій вибірці:
   Реальна ціна оренди  Прогнозована ціна оренди  Помилка
0                11500                   14825.0  -3325.0
1                 6500                    7314.0   -814.0
2                 6500                    2368.0   4132.0
3                43000                   41597.0   1403.0
4                 6500                    8934.0  -2434.0
5                 7000                   16735.0  -9735.0
6                 4000                    7332.0  -3332.0
7                 6000                    6158.0   -158.0
8                 8000                   11256.0  -3256.0
9                 7500                    9243.0  -1743.0


In [50]:
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_test_pred)

print("="*50)
print("МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):")
print("="*50)
print(f"\nMAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

mae = mean_absolute_error(y_train, y_train_pred)
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)
r2_train = r2_score(y_train, y_train_pred)

print("="*50)
print("МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тренувальній вибірці):")
print("="*50)
print(f"\nMAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2_train:.3f}")


МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):

MAE: 5060.09
RMSE: 7251.24
R²: 0.721
МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тренувальній вибірці):

MAE: 5160.34
RMSE: 7283.06
R²: 0.722


In [51]:
fig = px.scatter(
    x=y_test,
    y=y_test_pred,
    title='Реальна vs Прогнозована ціна оренди (тестова вибірка)',
    labels={'x': 'Реальна ціна оренди', 'y': 'Прогнозована ціна оренди'},
    opacity=0.6
)

max_val = max(y_test.max(), y_test_pred.max())
fig.add_trace(
    go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        name='Ідеальний прогноз',
        line=dict(color='red', dash='dash')
    )
)

fig.update_layout(height=500)
fig.show()

In [52]:
residuals = y_test - y_test_pred

fig = px.histogram(
    x=residuals,
    nbins=50,
    title='Розподіл помилок прогнозування',
    labels={'x': 'Помилка (реальні - прогнозовані)', 'count': 'Кількість'},
    color_discrete_sequence=['#e74c3c']
)
fig.add_vline(x=0, line_dash="dash", line_color="black", annotation_text="Ідеальний прогноз")
fig.update_layout(height=400)
fig.show()

In [53]:
fig = px.scatter(
    x=y_test_pred,
    y=residuals,
    title='Залежність помилок від прогнозованих значень',
    labels={'x': 'Прогнозована ціна оренди', 'y': 'Помилка'},
    opacity=0.5
)

fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Без помилки")

fig.update_layout(height=400)
fig.show()

In [54]:
errors_df = pd.DataFrame({
    'real': y_test.values,
    'predicted': y_test_pred,
    'error': np.abs(residuals)
})

top_errors = errors_df.nlargest(10, 'error')
print("Квартири з найбільшими помилками прогнозування:")
print(top_errors)

Квартири з найбільшими помилками прогнозування:
       real     predicted         error
1272  65000  29472.523430  35527.476570
2087  62000  30266.800111  31733.199889
1836  60000  29361.715246  30638.284754
608   20000  50121.738511  30121.738511
1908  65000  35388.881411  29611.118589
2387  65000  35898.810113  29101.189887
275   30000  59088.212921  29088.212921
4546  67000  38055.014179  28944.985821
2474  45000  16584.597575  28415.402425
1090  65000  38025.401767  26974.598233


In [55]:
errors_full = df_clean[['BHK', 'City', 'Furnishing Status']].loc[y_test.index].copy()

errors_full['predicted'] = y_test_pred
errors_full['abs_error'] = np.abs(residuals)
errors_full.head(3)

,BHK,City,Furnishing Status,predicted,abs_error
3767,2,Chennai,Semi-Furnished,14825.120789,3325.120789
1733,1,Bangalore,Semi-Furnished,7314.482201,814.482201
189,1,Kolkata,Furnished,2367.513038,4132.486962


In [56]:
errors_full.groupby('BHK')['abs_error'].mean().sort_values(ascending=False)

BHK
4    13298.694616
6    12375.329077
3     7558.602646
2     4502.962631
1     4023.501185
5     1248.592924
Name: abs_error, dtype: float64

In [57]:
errors_full.groupby('City')['abs_error'].mean().sort_values(ascending=False)

City
Mumbai       7964.705232
Delhi        6169.747221
Hyderabad    4503.754733
Bangalore    4274.250440
Chennai      4205.957018
Kolkata      4122.263302
Name: abs_error, dtype: float64

In [58]:
errors_full.groupby('Furnishing Status')['abs_error'].mean().sort_values(ascending=False)

Furnishing Status
Furnished         6010.577604
Semi-Furnished    5217.849617
Unfurnished       4580.496622
Name: abs_error, dtype: float64

### Висновок:  
Порівняно з початковою моделлю, значення R² на тестовій вибірці зросло з 0.693 до 0.721, що свідчить про підвищення здатності моделі пояснювати варіацію цільової змінної. Водночас різниця між train та test метриками зменшилась з 0.01 до 0.001, що вказує на покращення моделі без ознак перенавчання.  
Середні абсолютні помилки зменшились, що також свідчить про покращення точності прогнозування. Додавання нових ознак дозволило моделі краще враховувати відмінності між сегментами житла.  
Для подальшого підвищення якості моделі доцільно застосувати логарифмічне перетворення цільової змінної та протестувати моделі, що краще враховують нелінійні залежності.
